# The LOTlib3 library

> __**NOTE**__ This is based on Piantadosi's introduction to LOTlib3, which you can find [here](https://github.com/piantado/LOTlib3/blob/master/Documentation/Tutorial.md).

This week we're learning about Piantadosi's LOTlib3 library!

To run the code, let's first download the library into the folder:

In [1]:
try:
    # in colab
    import google.colab
    print('In colab, downloading LOTlib3')
    !git clone https://github.com/piantado/LOTlib3
except:
    # not in colab
    print('Not in colab!')

Not in colab!


## Imports

First we need to import a bunch of stuff:

In [2]:
import numpy as np

In [3]:
from LOTlib3.Miscellaneous import q, random
from LOTlib3.Grammar import Grammar
from LOTlib3.DataAndObjects import FunctionData, Obj
from LOTlib3.Hypotheses.LOTHypothesis import LOTHypothesis
from LOTlib3.Hypotheses.Priors.RationalRules import RationaRulesPrior
from LOTlib3.Hypotheses.Likelihoods.BinaryLikelihood import BinaryLikelihood
from LOTlib3.Eval import primitive
from LOTlib3.Miscellaneous import qq
from LOTlib3.TopN import TopN
from LOTlib3.Samplers.MetropolisHastings import MetropolisHastingsSampler